# Automated mesh extraction from .bin files

This notebook will detail an approach to taking the .bin files containing all the meshes in a programatic way, avoiding the CloudCompare GUI.

## CC command line extraction

The individual meshes can be extracted from the .bin files at the command line.
The filename saving options aren't the most flexible around the extraction of the meshes.

## Extracting all meshes at once.

Let's look first at extracting all the meshes to individual .ply files.

I've placed the `park_row.bin` merged mesh file in the Pointcept home dir one above this one.

In [16]:
import subprocess
import os
import shutil

from pathlib import Path


cloudcompare_path = "org.cloudcompare.CloudCompare"
input_path = "/home/sogilvy/park_row.bin"
input_stem = "park_row.bin"

# Make a clean working dir for this test.
# Also copy the input file there
workdir = "./test_all_meshes"
if os.path.exists(workdir):
    shutil.rmtree(workdir)
os.makedirs(workdir)
new_path = Path(workdir) / input_stem
shutil.copy(input_path, str(new_path))

# Define the command-line arguments to run with CloudCompare
# Example: Loading a file and performing an operation
command = [
    cloudcompare_path,
    "-SILENT",  # Run in silent mode (no GUI)
    "-O", input_stem,  # Load a mesh file (replace with your file path)
    "-M_EXPORT_FMT", "PLY",  # Set output format to PLY
    "-NO_TIMESTAMP", "-SAVE_MESHES",
    ]

try:
    # Run the command as a subprocess
    result = subprocess.run(command, cwd=workdir, check=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    # Print the output and any errors
    print("CloudCompare output:", result.stdout.decode())
    print("CloudCompare errors:", result.stderr.decode())
except subprocess.CalledProcessError as e:
    # Handle errors if the subprocess fails
    print(f"CloudCompare failed with error: {e.stderr.decode()}")


output_dir = Path(workdir)
for file in output_dir.iterdir():
    print(file.name)

CloudCompare output: [18:59:48] [ccColorScalesManager] Found 0 custom scale(s) in persistent settings
[18:59:48] [Plugin] Searching: /app/lib/cloudcompare/plugins
[18:59:48] 	Plugin found: Python Plugin (libPythonRuntime.so)
[18:59:48] 	Plugin found: 3DMASC (libQ3DMASC_PLUGIN.so)
[18:59:48] 	Plugin found: Additional I/O (libQADDITIONAL_IO_PLUGIN.so)
[18:59:48] 	Plugin found: Animation (libQANIMATION_PLUGIN.so)
[18:59:48] 	Plugin found: AutoSeg (libQAUTO_SEG_PLUGIN.so)
[18:59:48] 	Plugin found: CEA Virtual Broom (libQBROOM_PLUGIN.so)
[18:59:48] 	Plugin found: CANUPO (libQCANUPO_PLUGIN.so)
[18:59:48] 	Plugin found: Cloud layers (libQCLOUDLAYERS_PLUGIN.so)
[18:59:48] 	Plugin found: Colorimetric Segmenter (libQCOLORIMETRIC_SEGMENTER_PLUGIN.so)
[18:59:48] 	Plugin found: Compass (libQCOMPASS_PLUGIN.so)
[18:59:48] 	Plugin found: Core I/O (libQCORE_IO_PLUGIN.so)
[18:59:48] 	Plugin found: CSF Filter (libQCSF_PLUGIN.so)
[18:59:48] 	Plugin found: CSV Matrix I/O (libQCSV_MATRIX_IO_PLUGIN.so)
[18:5

The issue with this method is that the meshes are saved in the order in which they are listed in the default mesh file.
So the `4_CEILING` category which is listed first corresponds to `park_row_0.ply`.

We could just map these in the processing script, but any new files being put in would need added to the lookup table. Not a great solution.

## Intermediate lookup table

Instead, we can probably use the CloudCompare command line options to search for the appropriate mesh and if found, save it to a named file. Because the regex can match multiple entities in theory (though not in our examples), the only allowed file format is a .bin output from this stage.

This means we'll need to do an intermediate saving step, then we can run a similar command on those outputs as above to convert them into appropriately named .ply files.



In [23]:
# We need to iterate over the categories, allowing for some of them not existing
categories = [
    "1_WALL",
    "2_FLOOR",
    "3_ROOF",
]


# Make a clean working dir for this test.
# Also copy the input file there
workdir = "./test_lookup"
if os.path.exists(workdir):
    shutil.rmtree(workdir)
os.makedirs(workdir)
new_path = Path(workdir) / input_stem
shutil.copy(input_path, str(new_path))

# First loop over the 
for category in categories:
    command_regex = [
        cloudcompare_path,
        "-SILENT",
        "-O", input_stem,
        "-SELECT_ENTITIES",
        "-REGEX", category,
        "-RENAME_ENTITIES", category.lower(),
        "-NO_TIMESTAMP", "-SAVE_MESHES"
    ]
    
    try:
        # Run the command as a subprocess
        result = subprocess.run(command_regex, cwd=workdir, check=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        # output is long so skip printing it
        print("CloudCompare errors:", result.stderr.decode())
    except subprocess.CalledProcessError as e:
        print(f"CloudCompare failed with error: {e.stderr.decode()}")

# Cloudcompare appends an index suffix to renamed files when multiple entities are loaded
# So we'll strip this suffix for deterministic file names going forward.
# Regular expression to match filenames ending with '_<integer>.bin'
import re
pattern = re.compile(r"^(.*?_\d+)\.(bin)$")

for file_path in Path(workdir).iterdir():
    if file_path.is_file() and pattern.match(file_path.name):
        new_file_stem = re.sub(r'_\d+$', '', file_path.stem)
        new_file_path = file_path.with_name(f"{new_file_stem}{file_path.suffix}")
        file_path.rename(new_file_path)
        print(f"Renamed '{file_path.name}' to '{new_file_path.name}'")

        # Now convert the file to a .ply file for use outside of CloudCompare.
        command_convert = [
            cloudcompare_path,
            "-SILENT",
            "-O", new_file_path.name,
            "-M_EXPORT_FMT", "PLY",
            "-NO_TIMESTAMP", "-SAVE_MESHES",
        ]
        try:
            # Run the command as a subprocess
            result = subprocess.run(command_convert, cwd=workdir, check=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            # output is long so skip printing it
            print("CloudCompare errors:", result.stderr.decode())
        except subprocess.CalledProcessError as e:
            print(f"CloudCompare failed with error: {e.stderr.decode()}")

        # Finally unlink the intermediary .bin file
        new_file_path.unlink()

output_dir = Path(workdir)
for file in output_dir.iterdir():
    print(file.name)

CloudCompare errors: QSocketNotifier: Can only be used with threads started with QThread
libpng warning: iCCP: known incorrect sRGB profile

CloudCompare errors: QSocketNotifier: Can only be used with threads started with QThread
libpng warning: iCCP: known incorrect sRGB profile

CloudCompare errors: QSocketNotifier: Can only be used with threads started with QThread
libpng warning: iCCP: known incorrect sRGB profile

Renamed '1_wall_11.bin' to '1_wall.bin'
CloudCompare errors: QSocketNotifier: Can only be used with threads started with QThread
libpng warning: iCCP: known incorrect sRGB profile

Renamed '2_floor_3.bin' to '2_floor.bin'
CloudCompare errors: QSocketNotifier: Can only be used with threads started with QThread
libpng warning: iCCP: known incorrect sRGB profile

Renamed '3_roof_8.bin' to '3_roof.bin'
CloudCompare errors: QSocketNotifier: Can only be used with threads started with QThread
libpng warning: iCCP: known incorrect sRGB profile

park_row.bin
1_wall.ply
2_floor.pl

Opening these up in Cloudcompare shows the meshes seem to be properly saved as .ply entities.

This lets us now use a library like Open3D to process the meshes for our train/test/val splits without having to open the CloudCompare GUI a single time.